In [68]:
#---------------------------------------------------------------------------------------------------------------
# Nom du projet     : Classification
# Nom du programme  : SDA_Projet_Classification                                                                         
# Description       : L’objectif est de prédire le parti gagnant des élections présidentielles de 2020 aux Etats
#                   : Unis à partir de données socio-démographiques                        
# Auteur            : Yaya KONE,Papa Moussa THIOUNE                                                                         
# Date de création  : Novembre 2022                       
#---------------------------------------------------------------------------------------------------------------

# Librairies utiles pour le data management
import pandas as pd 
import numpy as np
import datetime as dt           
import os, glob, re  

#---------------------------------------------------------------------------------------------------------------
# Partie 1 : Constitution des donnes
#---------------------------------------------------------------------------------------------------------------

list_file_csv = glob.glob("data/*.csv")       # Liste des fichiers csv contenus dans le dossier data
list_file_xls = glob.glob("data/*.xls")       # Liste des fichiers xls contenus dans le dossier data 

# Importation des données
df_train = pd.read_csv('data/US_County_Level_Presidential_Results_08-16.csv')
df_test = pd.read_csv('data/2020_US_County_Level_Presidential_Results.csv')

train = pd.DataFrame(columns=['fips_code','year','dem','gop','oth'])
for i in range(2008,2017,4):
    df_train['year'] = i
    train = pd.concat([train, df_train[['fips_code','year','dem_'+str(i),'gop_'+str(i),'oth_'+str(i)]]\
                      .rename(columns={'dem_'+str(i): 'dem','gop_'+str(i):'gop','oth_'+str(i):'oth'})], axis=0)
    
    None

print(list_file_csv)
print(list_file_xls)
print(list_file)



['data/US_County_Level_Presidential_Results_08-16.csv', 'data/2020_US_County_Level_Presidential_Results.csv']
['data/PovertyEstimates.xls', 'data/PopulationEstimates.xls', 'data/Unemployment.xls', 'data/Education.xls']
None


In [69]:
train

,fips_code,year,dem,gop,oth
0,26041,2008,9974,8763,327
1,48295,2008,155,1093,8
2,1127,2008,7420,20722,510
3,48389,2008,1606,1445,26
4,56017,2008,619,1834,93
...,...,...,...,...,...
3107,17115,2016,18211,26782,2290
3108,29215,2016,1728,8875,332
3109,46051,2016,970,2381,211
3110,17103,2016,5499,8597,1119


In [46]:
i=2008

col_dem = "dem_" + str(i)
train = df_train[['fips_code','dem_'+str(i),'gop_'+str(i),'oth_'+str(i)]]

In [47]:
train

,fips_code,dem_2008,gop_2008,oth_2008
0,26041,9974,8763,327
1,48295,155,1093,8
2,1127,7420,20722,510
3,48389,1606,1445,26
4,56017,619,1834,93
...,...,...,...,...
3107,17115,25487,24948,781
3108,29215,3410,7215,226
3109,46051,1786,1951,93
3110,17103,7765,8258,295


In [50]:
for i in range(2008,2017,4):
    print(i)

2008
2012
2016
